# Introduction/Business Problem

#### Lahore is one of the biggest metroploitan cities in Pakistan. It is vibrant, densly populated and presents a welcoming environment for aspiring entreprenuers. With the advent of the digital age one of the most viable mediums to start a business are proving to be digital platforms. There are hundreds of business in lahore that make use of such platforms but are yet unable to sustain their business because they are unable to reach the intended audience. On the other hand , we also hear about the success stories of start ups taking off and breaking even within their first few months with the help of targeted advertisements on these platforms. In a busy metroplitan like Lahore it is well established that time is money and every week hours of that precious time and fuel are wasted stuck in traffic on the way to do a routine errand like grocery shopping. The aim of this project would be to help online grocery stores target neighbourhoods that have the least number of shopping centers so that not only they save cost on advertisement but have a more effective penetration than a random carpet advertisement. In this project we will assume that our client is a startup that delivers groceries to your doorstep and is looking to increase its service penetration in Lahore while at the same time not overspending on advertisement. We will run a cluster analysis on neoghbourhoods of Lahore and identify with reasoning the neighbourhoods that are expected to be more reactive to the services offered by the startup as compared to others.


## Data

#### The data I will be using in this project is in three parts 
#### 1) List of town in Lahore (Source of data :https://en.wikipedia.org/wiki/List_of_towns_in_Lahore)
##### Example Row

In [1]:
from lxml import etree
import urllib

In [2]:
import requests
import pandas as pd
url = 'https://en.wikipedia.org/wiki/List_of_towns_in_Lahore'
html = requests.get(url).content
df_list = pd.read_html(html)
df = df_list[-1]

In [13]:
#List of towns
df.iloc[1,1]

'Kot Begum (UC 1) Yousaf Bagh (UC 2) Kot Kamboh (UC 3) Shamsabad (UC 4) Chah Cheemay Walay (UC 5) Shahrak-e-Aziz (UC 6) Lajpat Nagar (UC 7) Faisal Bagh (UC 8) Javed Bagh (UC 9) Shahrak-e-Qaisar (UC 05) Majeed Bagh (UC 11) Qazi Bagh (UC 12) Ravi-Clifton (UC 13) Ladhay Shah (UC 14) Qila Lachman Singh (UC 15) Shahrak-e-Auqaf (UC 16) Farooqganj (UC 17) Hanif Bagh (UC 18) Siddiqpue Pura (UC 19) Shahrak-e-Larex (UC 20) Shahrak-e-Badar (UC 21) Data Nagar (UC 22) Shahrak-e-Saddiqua (UC 23) Bhama Jhugian (UC 25) Usmanganj (UC 29) Manzorabad (UC 30) Mochi Gate (UC 31) Azam Markaz (UC 33) Shah Alam Markaz (UC 34) Rang Mahal, Lahore (UC 35) Lohari Gate (UC 36) Bhati Gate (UC 37) Shahi Qila (UC 38) Sotar Mandi (UC 39)'

#### 2) Coordinates of towns in Lahore (Source of data: geopy library in python)

In [15]:
!conda install -c conda-forge geopy --yes

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.16.0-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00 529.33 kB/s
geopy-1.16.0-p 100% |################################| Time: 0:00:00 815.81 kB/s


In [18]:
from geopy.geocoders import Nominatim
address = 'Mochi Gate, Lahore'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address,latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/geopy/geocoders/osm.py:138: UserWarning: Using Nominatim with the default "geopy/1.16.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Mochi Gate, Lahore are 31.5769498, 74.3215639.


### 3)Nearby Venues and their Categories ( Source of data : Foursquare Library API)

In [19]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: SNYON02DZSU3M1W2ETYHYVW31M5IWLFYV3J0VV0LOJU15B2M
CLIENT_SECRET:45LXYSQ4XBNVLXJY3BO4IKPMTCLW1S04I12I54VTQXLTZFIB


In [20]:
neighborhood_latitude= latitude
neighborhood_longitude = longitude
radius = 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

url

'https://api.foursquare.com/v2/venues/explore?&client_id=SNYON02DZSU3M1W2ETYHYVW31M5IWLFYV3J0VV0LOJU15B2M&client_secret=45LXYSQ4XBNVLXJY3BO4IKPMTCLW1S04I12I54VTQXLTZFIB&v=20180602&ll=31.5769498,74.3215639&radius=500&limit=100'

In [22]:
results = requests.get(url).json()
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [24]:
from pandas.io.json import json_normalize
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Gawalmandi,Neighborhood,31.572570,74.322060
1,Mochi Gate موچی دروازہ (موچی دروازہ),Historic Site,31.577183,74.321544
2,Karachi Pipe Store,Construction & Landscaping,31.576735,74.323704
3,Saien Shahi Kabab,BBQ Joint,31.578882,74.324949
4,Sadiq Halwa Poori,Breakfast Spot,31.574209,74.324347
